In [ ]:
!pip freeze | grep tensorflow

In [ ]:
# pip install tensorflow==1.9.0rc2 coremltools==0.8 tensorflowjs==0.5.0
# pip install tf-nightly==1.10.0.dev20180707
# nano /usr/local/anaconda3/envs/tf2/lib/python3.6/site-packages/tensorflow/contrib/lite/python/lite.py

In [ ]:
#pip install coremltools==0.8 Keras==2.1.6 imutils==0.4.6 opencv-python==3.4.1.15


In [ ]:
#https://github.com/ianlokh/MobileNet-On-CoreML.git

In [1]:
import os
import shutil
import hashlib
import urllib.request
import numpy as np
import json
import keras
import tensorflow as tf
import coremltools
import tensorflowjs as tfjs
from tensorflow.python import keras as tf_keras

/usr/local/anaconda3/envs/tf2/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:

#from keras import backend as K
#from keras.models import load_model
#from keras.utils.generic_utils import CustomObjectScope
#from keras.applications.mobilenet import preprocess_input

#from keras.preprocessing import image
#from keras.applications.mobilenet import preprocess_input



In [ ]:
!ls

In [2]:
with open('labels.txt') as f:
    labels = f.read().splitlines()

In [3]:
labels_dict = {idx:label for (idx,label) in enumerate(labels)}

In [4]:
labels_dict

{0: 'scottish_deerhound',
 1: 'maltese_dog',
 2: 'afghan_hound',
 3: 'entlebucher',
 4: 'bernese_mountain_dog',
 5: 'shih-tzu',
 6: 'great_pyrenees',
 7: 'pomeranian',
 8: 'basenji',
 9: 'samoyed',
 10: 'airedale',
 11: 'tibetan_terrier',
 12: 'leonberg',
 13: 'cairn',
 14: 'beagle',
 15: 'japanese_spaniel',
 16: 'australian_terrier',
 17: 'blenheim_spaniel',
 18: 'miniature_pinscher',
 19: 'irish_wolfhound',
 20: 'lakeland_terrier',
 21: 'saluki',
 22: 'papillon',
 23: 'whippet',
 24: 'siberian_husky',
 25: 'norwegian_elkhound',
 26: 'pug',
 27: 'chow',
 28: 'italian_greyhound',
 29: 'pembroke',
 30: 'ibizan_hound',
 31: 'border_terrier',
 32: 'newfoundland',
 33: 'lhasa',
 34: 'silky_terrier',
 35: 'bedlington_terrier',
 36: 'dandie_dinmont',
 37: 'irish_setter',
 38: 'sealyham_terrier',
 39: 'rhodesian_ridgeback',
 40: 'old_english_sheepdog',
 41: 'collie',
 42: 'boston_bull',
 43: 'english_foxhound',
 44: 'bouvier_des_flandres',
 45: 'african_hunting_dog',
 46: 'schipperke',
 47: '

In [5]:
json.dumps(labels_dict)

'{"0": "scottish_deerhound", "1": "maltese_dog", "2": "afghan_hound", "3": "entlebucher", "4": "bernese_mountain_dog", "5": "shih-tzu", "6": "great_pyrenees", "7": "pomeranian", "8": "basenji", "9": "samoyed", "10": "airedale", "11": "tibetan_terrier", "12": "leonberg", "13": "cairn", "14": "beagle", "15": "japanese_spaniel", "16": "australian_terrier", "17": "blenheim_spaniel", "18": "miniature_pinscher", "19": "irish_wolfhound", "20": "lakeland_terrier", "21": "saluki", "22": "papillon", "23": "whippet", "24": "siberian_husky", "25": "norwegian_elkhound", "26": "pug", "27": "chow", "28": "italian_greyhound", "29": "pembroke", "30": "ibizan_hound", "31": "border_terrier", "32": "newfoundland", "33": "lhasa", "34": "silky_terrier", "35": "bedlington_terrier", "36": "dandie_dinmont", "37": "irish_setter", "38": "sealyham_terrier", "39": "rhodesian_ridgeback", "40": "old_english_sheepdog", "41": "collie", "42": "boston_bull", "43": "english_foxhound", "44": "bouvier_des_flandres", "45": 

In [6]:
import json
with open('label_classes.json', 'w') as fp:
    json.dump(labels_dict, fp, indent=4 )

In [ ]:
labels

In [7]:
def download_image(url,ext=".jpg"):
    url_hash = hashlib.md5(url.encode('utf-8')).hexdigest()
    file_path = f'{DOWNLOAD_FOLDER}/{url_hash}.{ext}'
    
    urllib.request.urlretrieve(url,file_path)

    img = tf_keras.preprocessing.image.load_img(file_path, target_size=(224, 224))
    return img

def predict(im, model, labels,n=5):
    out = model.predict(img_arr[None,:,:,:]).flatten()
    top_indices = out.argsort()[::-1][:n]
    
    top_probability = out[top_indices]
    top_labels = np.array(labels)[top_indices]
    
    res = list(zip(top_labels,top_probability))
    
    return res

def get_top_n_predictions(preds,labels,n=5):
    if len(preds.shape) >1:
        preds = preds.flatten()
    top_idxs = (-preds).argsort()[:n]
    res =[]
    for idx in top_idxs:
        res.append((labels[idx],preds[idx] ))
    return res

In [8]:
DOWNLOAD_FOLDER='downloads'

In [9]:
shutil.rmtree(DOWNLOAD_FOLDER,ignore_errors=True)
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

In [ ]:
img_url = "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231410/Labrador-Retriever-On-White-01.jpg"
img = download_image(img_url)
img


In [ ]:
img_preprocessed = keras.applications.mobilenet.preprocess_input (
        tf_keras.preprocessing.image.img_to_array(img)
)

In [ ]:
img_arr = keras.applications.mobilenet.preprocess_input (
        tf_keras.preprocessing.image.img_to_array(img)
)

# Keras

In [ ]:
def load_keras_model(model_path):
    with keras.utils.generic_utils.CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
        keras_model =  keras.models.load_model(model_path)
    return keras_model

In [ ]:
keras_model = load_keras_model(model_path='dog_breed.h5')


In [ ]:
predict(im=img_preprocessed ,model=keras_model, labels=labels)

# Tensorflow Estimator

In [10]:
from tensorflow.python import keras as tf_keras

In [ ]:
#from tensorflow.python.keras.models import load_model


In [ ]:
keras.applications.mobilenet.relu6

In [ ]:
def load_keras_using_tf(model_path):

    custom_objects={'relu6': tf_keras.applications.mobilenet.relu6,'DepthwiseConv2D': tf_keras.applications.mobilenet.DepthwiseConv2D}
    
    model = tf_keras.models.load_model(model_path,custom_objects=custom_objects)
        
    mobilenet_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, custom_objects={'relu6': tf_keras.applications.mobilenet.relu6})
    
    return mobilenet_estimator

In [ ]:
tf_keras_model = load_keras_using_tf('dog_breed.h5')

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"input_1": img_preprocessed[None,:,:,:]},
  num_epochs=1,
  shuffle=False)

In [ ]:
predictions = list(tf_keras_model.predict(input_fn=predict_input_fn))
preds= predictions[0]['dense_1']
get_top_n_predictions(preds=preds,labels=labels)

# convert tfjs

In [ ]:
!pip install tensorflowjs==0.5.0

In [ ]:
?tfjs.converters.save_keras_model

In [ ]:
tfjs.converters.save_keras_model(
    model = keras_model, 
    artifacts_dir='tfjs_artifacts')

# convert tflite

In [ ]:
?tf.contrib.lite.TocoConverter.from_keras_model_file

In [11]:
with tf_keras.utils.generic_utils.CustomObjectScope({'relu6': tf_keras.applications.mobilenet.relu6,'DepthwiseConv2D': tf_keras.applications.mobilenet.DepthwiseConv2D}):
    converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed.h5")
    
    tflite_model = converter.convert()
    open("dog_breed.tflite", "wb").write(tflite_model)

INFO:tensorflow:Froze 137 variables.


INFO:tensorflow:Froze 137 variables.


INFO:tensorflow:Converted 137 variables to const ops.


INFO:tensorflow:Converted 137 variables to const ops.


RuntimeError: TOCO failed see console for info.
b'2018-07-11 07:35:06.749379: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before Removing unused ops: 398 operators, 632 arrays (0 quantized)\n2018-07-11 07:35:06.762622: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before general graph transformations: 398 operators, 632 arrays (0 quantized)\n2018-07-11 07:35:06.788503: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] After general graph transformations pass 1: 98 operators, 224 arrays (0 quantized)\n2018-07-11 07:35:06.791154: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before dequantization graph transformations: 98 operators, 224 arrays (0 quantized)\n2018-07-11 07:35:06.793722: I tensorflow/contrib/lite/toco/allocate_transient_arrays.cc:329] Total transient array allocated size: 8028160 bytes, theoretical optimal value: 6538240 bytes.\n2018

In [12]:
with tf_keras.utils.generic_utils.CustomObjectScope({'relu6': tf_keras.applications.mobilenet.relu6,'DepthwiseConv2D': tf_keras.applications.mobilenet.DepthwiseConv2D}):
    converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed_tf.h5")
    
    tflite_model = converter.convert()
    open("dog_breed.tflite", "wb").write(tflite_model)

INFO:tensorflow:Froze 137 variables.


INFO:tensorflow:Froze 137 variables.


INFO:tensorflow:Converted 137 variables to const ops.


INFO:tensorflow:Converted 137 variables to const ops.


RuntimeError: TOCO failed see console for info.
b'2018-07-11 07:36:26.041045: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before Removing unused ops: 398 operators, 632 arrays (0 quantized)\n2018-07-11 07:36:26.053084: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before general graph transformations: 398 operators, 632 arrays (0 quantized)\n2018-07-11 07:36:26.078778: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] After general graph transformations pass 1: 98 operators, 224 arrays (0 quantized)\n2018-07-11 07:36:26.081666: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before dequantization graph transformations: 98 operators, 224 arrays (0 quantized)\n2018-07-11 07:36:26.084547: I tensorflow/contrib/lite/toco/allocate_transient_arrays.cc:329] Total transient array allocated size: 8028160 bytes, theoretical optimal value: 6538240 bytes.\n2018

In [ ]:
with keras.utils.generic_utils.CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed.h5")
    tflite_model = converter.convert()
    open("dog_breed.tflite", "wb").write(tflite_model)



# coreml

https://developer.apple.com/documentation/vision/classifying_images_with_vision_and_core_ml

In [ ]:
scale = 1./255
coreml_model = coremltools.converters.keras.convert(keras_model,
                                                    input_names = 'image',
                                                    output_names = 'output',
                                                    image_input_names = 'image',
                                                    image_scale = scale,
                                                    red_bias = -123.68 * scale,
                                                    green_bias = -116.779 * scale,
                                                    blue_bias = -103.939 * scale,
                                                    class_labels=labels)

coreml_model.author = 'Ian Lo'
coreml_model.license = 'MIT'
coreml_model.short_description = 'Model to classify 17 different Singaporean dishes'
coreml_model.input_description['image'] = 'Images from camera in CVPixelBuffer'
coreml_model.output_description['output'] = 'Predicted dishes'

coreml_model.save('dog_breed.mlmodel')

In [ ]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import models
from tensorflow.python.keras import layers


In [ ]:
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils import CustomObjectScope
def relu6(x):
    return K.relu(x, max_value=6)
with CustomObjectScope({'relu6': relu6}):
    keras_mobilenet= tf.keras.models.load_model(filepath='dog_breed.h5')

    mobilenet_estimator = tf.keras.estimator.model_to_estimator(keras_model=keras_mobilenet,  custom_objects={'relu6': relu6})

In [ ]:
??preprocess_input

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"input_1": preprocess_input(image.img_to_array(img))[None,:,:,:]},
  num_epochs=1,
  shuffle=False)

In [ ]:
predictions = list(mobilenet_estimator.predict(input_fn=predict_input_fn))
#predicted_classes = [p["classes"] for p in predictions]

In [ ]:
preds= predictions[0]['dense_1']
get_top_n_predictions(preds=preds,labels=labels)

In [ ]:
predictions = list(mobilenet_estimator.predict(input_fn=predict_input_fn))
preds= predictions[0]['dense_1']
get_top_n_predictions(preds=preds,labels=labels)

In [ ]:
def serving_input_receiver_fn():
    inputs = {
        'input_1': tf.placeholder(tf.float32, [None, 224, 224, 3]),
      }
    return tf.estimator.export.ServingInputReceiver(inputs)


In [ ]:
mobilenet_estimator.export_savedmodel('tmp',serving_input_receiver_fn=serving_input_receiver_fn)

In [ ]:
?tf.estimator.export.ServingInputReceiver

In [ ]:
?mobilenet_estimator.export_savedmodel

In [ ]:
!ls tmp/1531058391

In [ ]:
?tf.estimator.export.ServingInputReceiver

In [ ]:
from te.contrib.lite import TocoConverter

In [ ]:
!pwd

In [ ]:
!ls dog_breed.h5 

In [ ]:
tf.contrib.lite.TocoConverter.

In [ ]:
?converter.convert

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import mobilenet

In [ ]:
from tensorflow.python.keras.utils.generic_utils import CustomObjectScope
from tensorflow.python.keras.applications import mobilenet
from tensorflow.python.keras.models import load_model
with CustomObjectScope({'relu6': mobilenet.relu6,'DepthwiseConv2D': mobilenet.DepthwiseConv2D}):
    model = load_model('dog_breed.h5')
    converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed.h5")
    
    tflite_model = converter.convert()
    open("dog_breed.tflite", "wb").write(tflite_model)

In [ ]:
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    model = keras.models.load_model('dog_breed.h5')


Because the Keras model was trained on ImageNet, need to subtract the mean from RGB

- red_bias=-123.68
- green_bias=-116.78
- blue_bias=-103.94

In [ ]:
scale = 1./255
coreml_model = coremltools.converters.keras.convert(model,
                                                    input_names = 'image',
                                                    output_names = 'output',
                                                    image_input_names = 'image',
                                                    image_scale = scale,
                                                    red_bias = -123.68 * scale,
                                                    green_bias = -116.779 * scale,
                                                    blue_bias = -103.939 * scale,
                                                    class_labels=labels)

coreml_model.author = 'Ian Lo'
coreml_model.license = 'MIT'
coreml_model.short_description = 'Model to classify 17 different Singaporean dishes'
coreml_model.input_description['image'] = 'Images from camera in CVPixelBuffer'
coreml_model.output_description['output'] = 'Predicted dishes'

coreml_model.save('dog_breed.mlmodel')

In [ ]:
!ls 

In [ ]:
? converter.convert

In [ ]:
with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    model =  load_model('dog_breed.h5')



In [ ]:
import tensorflow as tf

converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed.h5")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
tf.__version__


In [ ]:
import tensorflow as tf

converter = tf.contrib.lite.TocoConverter.from_keras_model_file('dog_breed.h5')
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
tf.__version__

In [ ]:
converter = tf.contrib.lite.TocoConverter.from_keras_model_file('dog_breed.h5')
